In [1]:
import numpy as np
import skimage
import utils
import pathlib
import matplotlib.pyplot as plt

In [2]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # Compute the normalized histogram
    intensity_range = (im.max() - im.min()) + 1                           
    H, W = im.shape
    hist, bins = np.histogram(im.flatten(), bins=intensity_range)
    norm_hist = hist/(H * W)

    # Compute the cumulative sums
    cumulative_sums = np.cumsum(norm_hist)

    # Compute the cumulative means
    cumulative_means = np.zeros(intensity_range)
    for i in range(1, intensity_range):
        cumulative_means[i] = cumulative_means[i - 1] + i * norm_hist[i]
        
    # Compute the global means
    global_mean = 0
    for i in range(intensity_range):
        global_mean += i * norm_hist[i]

    # Between-class variances
    variances = ((global_mean * cumulative_sums -
                               cumulative_means)**2 + 1e-32) / ((cumulative_sums * (1 - cumulative_sums)) + 1e-32)

    # Obtain the Otsu threshold as the average of between-class variances maximum values 
    max_indices = np.argwhere(variances == max(variances))
    threshold = max_indices.item() if len(max_indices) == 1 else sum(max_indices)/len(max_indices)

    # Shift the threshold by im.min()
    threshold += im.min()

    return threshold
    ### END YOUR CODE HERE ###

In [3]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images/thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed/thumbprint-segmented.png
Reading image: images/rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed/rice-shaded-segmented.png
